# SNLP Assignment 6 - Text Classification

Name 1: William LaCroix<br/>
Student id 1: 7038732<br/>
Email 1: williamplacroix@gmail.com<br/>


Name 2: Nicholas Jennings<br/>
Student id 2: 2573492<br/>
Email 2: s8nijenn@stud.uni-saarland.de<br/>

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the respective Python files for any additional exercises necessary. There is no need to submit the data files should they exist. <br/>
Upload the zipped folder on CMS. Only one member of the group should make the submisssion.

---

## <span style="color:red">Sentiment Analysis</span>

   - Use the [FinancialPhraseBank](https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news) corpus with a 80:20 train test split, with a) a naive bayes b) XGBoost classifier for sentiment analysis (**2 points**)

      - Naive Bayes: https://scikit-learn.org/stable/modules/naive_bayes.html

      - XGBoost: https://xgboost.readthedocs.io/en/stable/index.html

      - <span style="color:red">Note:</span> Only use all_data.csv file for your experiments

   - Use [LoughranMcDonald master dictionary](https://drive.google.com/file/d/17CmUZM9hGUdGYjCXcjQLyybjTrcjrhik/view?usp=sharing) to get word level polarity, replace words with the their corresponding polarity and now repeat the same classification task as above (**4 points**)

      - What happens if you remove the stop words? 

      - What's the ratio of stopwords (taken from NLTK) amongst the total word count? 

      - Is it a good choice to do stopword removal? Explain with 2-3 examples why or why not?
   
   - Create cascaded Bi-Grams and tri-grams and repeat both the exercises from above. Which representation do you think is a better value in terms of accuracy and computational power required? (**3 points**)

      - Bi-grams example: 

         - Company A barely surpassed their profit expectations.
         
         - (Company, A), (A, barely), (barely, surpassed), (surpassed, their) ...
   
   - How would you represnt the polarity in uni/bi/tri-gram models with Huffman Encoding? (**1 point**)


### Few points to remember
   - While splitting your dataset, use seed=42
   - Use type hint(s) in your code, and add a docstring to your functions or classes
   - Focus on the readability of your code, that helps us to give you better feedback on where the code went wrong
   - <span style="color:red">Do not submit the data or dictionary file.</span>

### Discussion:

- What happens if you remove the stop words?
    - Answer:

- What's the ratio of stopwords (taken from NLTK) amongst the total word count? 
    - Answer:

- Is it a good choice to do stopword removal? Explain with 2-3 examples why or why not?
    - Answer:
  
- How would you represnt the polarity in uni/bi/tri-gram models with Huffman Encoding? (**1 point**)
    - Answer:


In [8]:
import solution
from importlib import reload
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb


In [23]:
# 1. Unigram Naive Bayes, unpolarized -> baseline NB
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file, since it's a multi-class classification
# Confusion matrix should give you a better idea of how well your model is performing
# Naive Bayes classification of 3 class labels
# NB_confusion_matrix on 3x3 matrix
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Baseline unpolairzed unigram")


          0                                                  1
0   neutral  according to gran the company has no plans to ...
1   neutral  technopolis plans to develop in stages an area...
2  negative  the international electronic industry company ...
3  positive  with the new production plant the company woul...
4  positive  according to the company s updated strategy fo...


KeyboardInterrupt: 

In [21]:
# 2. Unigram XGBoost, unpolarized -> baseline XGBoost
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = solution.load_and_preprocess_data()

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Baseline unpolairzed unigram")

Predicted   0    1    2  All
True                        
0          58   40   10  108
1          16  543   36  595
2          10   94  163  267
All        84  677  209  970

Unpolairzed unigram XGBoost accuracy: 0.5893333333333334
Unpolairzed unigram XGBoost precision: 0.6104868913857678
Unpolairzed unigram XGBoost recall: 0.7652582159624414
Unpolairzed unigram XGBoost F1: 0.6791666666666667


In [ ]:
# 3. Unigram Naive Bayes, polarized, include stop words
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

#TODO polarize corpus

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Polairzed unigram with stop words")

In [ ]:
# 4. Unigram Naive Bayes, polarized, exclude stop words
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

#TODO remove stop words

#TODO polarize corpus

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Polairzed unigram without stop words")

In [ ]:
# 5. Unigram XGBoost, polarized, include stop words
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = solution.load_and_preprocess_data()

#TODO polarize corpus

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Polairzed unigram with stop words")

In [ ]:
# 6. Unigram XGBoost, polarized, exclude stop words
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = solution.load_and_preprocess_data()

#TODO remove stop words

#TODO polarize corpus

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Polairzed unigram without stop words")

In [ ]:
# 7. Bigram Naive Bayes, polarized, include stop words
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

#TODO polarize corpus

#TODO bigramize corpus

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Polairzed bigram with stop words")

In [ ]:
# 8. Bigram Naive Bayes, polarized, exclude stop words
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

#TODO remove stop words

#TODO polarize corpus

#TODO bigramize corpus

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Polairzed bigram without stop words")

In [ ]:
# 9. Bigram XGBoost, polaraized, include stop words
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = solution.load_and_preprocess_data()

#TODO polarize corpus

#TODO bigramize corpus

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Polairzed bigram with stop words")


In [ ]:
# 10. Bigram XGBoost, polarized, exclude stop words
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = solution.load_and_preprocess_data()

#TODO remove stop words

#TODO polarize corpus

#TODO bigramize corpus

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Polairzed bigram without stop words")

In [ ]:
# 11. Trigram Naive Bayes, polarized, include stop words
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

#TODO polarize corpus

#TODO trigramize corpus

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Polairzed trigram with stop words")

In [ ]:
# 12. Trigram Naive Bayes, polarized, exclude stop words
solution = reload(solution)
classification_model = MultinomialNB()
corpus = solution.load_and_preprocess_data()

#TODO remove stop words

#TODO polarize corpus

#TODO bigramize corpus

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="Naive Bayes", preprocessing="Polairzed bigram without stop words")

In [ ]:
# 13. Trigram XGBoost, polarized, include stop words
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = corpus.copy()

#TODO polarize corpus

#TODO trigramize corpus

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Polairzed trigram with stop words")

In [ ]:
# 14. Trigram XGBoost, polarized, exclude stop words
solution = reload(solution)
classification_model = xgb.XGBClassifier()
corpus = corpus.copy()

#TODO remove stop words

#TODO polarize corpus

#TODO trigramize corpus

# replace labels with numbers for XGBoost classifier
corpus[0] = corpus[0].replace("positive", 2)
corpus[0] = corpus[0].replace("neutral", 1)
corpus[0] = corpus[0].replace("negative", 0)

y_test, y_pred = solution.train_and_fit_model(corpus, classification_model)

# Get the confusion matrix from solution.py file
confusion_matrix = solution.confusion_matrix(y_test, y_pred)
print(confusion_matrix)
solution.test(confusion_matrix, classifier="XGBoost", preprocessing="Polairzed trigram without stop words")